In [1]:
import pandas as pd
import numpy as np
%matplotlib inline 
import spacy
import re
from functools import reduce
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn import ensemble
from sklearn.model_selection import train_test_split

from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


height has been deprecated.



In [2]:
#Load English Spacy
nlp = spacy.load("en")

In [3]:
# Data obtained from Figure Eight.  Contains economic news articles that were categorized as either having information on the US economy or not.
data = pd.read_csv("Economic-News.csv",encoding = "ISO-8859-1")
#link: https://www.figure-eight.com/wp-content/uploads/2016/03/Full-Economic-News-DFE-839861.csv

In [4]:
# See a few rows of the data
data.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,positivity,positivity:confidence,relevance,relevance:confidence,articleid,date,headline,positivity_gold,relevance_gold,text
0,842613455,False,finalized,3,12/5/15 17:48,3.0,0.6400,yes,0.640,wsj_398217788,8/14/91,Yields on CDs Fell in the Latest Week,NaN,NaN,NEW YORK -- Yields on most certificates of dep...
1,842613456,False,finalized,3,12/5/15 16:54,NaN,NaN,no,1.000,wsj_399019502,8/21/07,The Morning Brief: White House Seeks to Limit ...,NaN,NaN,The Wall Street Journal Online</br></br>The Mo...
2,842613457,False,finalized,3,12/5/15 1:59,NaN,NaN,no,1.000,wsj_398284048,11/14/91,Banking Bill Negotiators Set Compromise --- Pl...,NaN,NaN,WASHINGTON -- In an effort to achieve banking ...
3,842613458,False,finalized,3,12/5/15 2:19,NaN,0.0000,no,0.675,wsj_397959018,6/16/86,Manager's Journal: Sniffing Out Drug Abusers I...,NaN,NaN,The statistics on the enormous costs of employ...
4,842613459,False,finalized,3,12/5/15 17:48,3.0,0.3257,yes,0.640,wsj_398838054,10/4/02,Currency Trading: Dollar Remains in Tight Rang...,NaN,NaN,NEW YORK -- Indecision marked the dollar's ton...


In [5]:
# See data shape
data.shape

(8000, 15)

In [6]:
# Check for imbalance
data["relevance"].value_counts()


no          6571
yes         1420
not sure       9
Name: relevance, dtype: int64

In [7]:
#Separate data into relevant and not relevant for parsing
relevant = data[data["relevance"] == "yes"]
not_relevant = data[data["relevance"] == "no"]
relevant = relevant.iloc[0:500,]
not_relevant = not_relevant.iloc[0:500,]
yes_text = relevant["text"]
no_text = not_relevant["text"]

In [8]:
#Create list to see samples of data to determine what cleaning is needed
text= data["text"]
relevance = data["relevance"]
text_list= (1,100,500,1000,1500,2250,2500,3000,3350,3400,4500)

In [9]:
# See text to determine cleaning needed.
for headline in text_list:
    print (text[headline][:]) #Remove slice to see full data while determining what cleaning is needed

The Wall Street Journal Online</br></br>The Morning Brief, a look at the day's biggest news, is emailed to subscribers by 7 a.m. every business day. Sign up for the e-mail here.</br></br>On Friday evening, with Congress out of town on its summer recess and Americans heading into a mid-August weekend, the Bush administration sent a message to the states: The federal government will make it tougher for a national children's insurance program to cover the offspring of middle-income families.</br></br>The State Children's Health Insurance Program was created in 1997 to help children whose families couldn't afford insurance but didn't qualify for Medicaid, and administration officials tell the New York Times that the changes are aimed at returning the program to its low- income focus and assuring it didn't become a replacement for private insurance. Administration point man Dennis Smith wrote to state officials saying there would be new restrictions on the District of Columbia and the 18 st

It's no secret that the tea party faithful regard the Obama administration as a Constitution-shredding tyranny. But in a profile of the movement published last week, the New York Times reported the surprising news that many of the protesters have come to this view as a result of their experiences in the recession: "Their families upended by lost jobs, foreclosed homes and depleted retirement funds, they said they wanted to know why it happened and whom to blame."</br></br>An account of a conservative uprising published a few days later by the Washington Post under the headline, "Appalachia is slipping from grip of Democrats," told the story of a hard-bitten congressional district in western Virginia where the response to the recession has been a dramatic swerve to the right.</br></br>The free-market system blunders into recession; its victims flock to the free-market banner. And here we go again.</br></br>The backlash against liberalism has been going on for more than 40 years. It is a

Regan, in his strongest language to date, blamed banks and other financial institutions yesterday for keeping interest rates high while other sectorsof the economy were helping to reduce inflation.</br></br>Regan, insisting that the large federal budget deficits were not responsible for the persistent high interest rates,,, said, there doesnÛªt ÛÏseem to be a good reason-ÛÓor should I say there is no good excuseÛ for those rates to be'.as high as they are, especially with ihe recent declines in inflation.</br></br>He said that if banks set their charges more in line with the declining inflation rateÛÓwhich has dropped below 4 percent from about 13 percent at the start of the Reagan administrationÛÓinterest rates would come down.</br></br>ÛÏOther major sections of the American economy are now moving in accord with the current environment of low inflation,Û Regan told a luncheon of the Chemical Manufacturers Association. ÛÏAnd sooner, I hope, rather than later, the banks and o

In [10]:
# Clean the text
def text_cleaner(text):
    text = text.apply(lambda x:re.sub("</br></br>", " ", x))
    text = text.apply(lambda x:re.sub("---", " ", x))
    text = text.apply(lambda x:re.sub("--", " ", x))
    text = text.apply(lambda x:re.sub("<", " ", x))
    text = text.apply(lambda x:re.sub(";", " ", x))
    text = text.apply(lambda x:re.sub(";", " ", x))
    text = text.apply(lambda x:re.sub("ÛÏ", " ", x))
    text = text.apply(lambda x:re.sub("Ûª", " ", x))
    text = text.apply(lambda x:re.sub("Û", " ", x))
    text = text.apply(lambda x:re.sub("ûó", " ", x))
    text = text.apply(lambda x:re.sub("ÛÓ", " ", x))
    text = text.apply(lambda x:re.sub("\x89ûó", " ", x))
    text = text.apply(lambda x:re.sub("x89ûó", " ", x))
    text = text.apply(lambda x:re.sub("\x89ðê", " ", x))  
    text = text.apply(lambda x:re.sub("Author:", " ", x))
    text = text.apply(lambda x:re.sub("By:", " ", x))
    text = text.apply(lambda x:re.sub("\d+","",x))    
    text = text.apply(lambda x:re.sub("'","",x)) 
    text = text.apply(lambda x:re.sub("  ","",x)) 
    text = text.apply(lambda x:re.sub("\\\\","",x))  
    text = text.apply(lambda x:re.sub("/","",x))  
    text = text.apply(lambda x:re.sub("-", " ", x))
    text = text.apply(lambda x:re.sub("\+", " ", x))
    return text
cleaned_text=text_cleaner(text)
yes_cleaned = text_cleaner(yes_text)
no_cleaned = text_cleaner(no_text)
    

In [11]:
# See cleaned text and check if additional cleaning needed
for headline in text_list:
    print (cleaned_text[headline][:500]) #Remove slice if needed to see full data

The Wall Street Journal Online The Morning Brief, a look at the days biggest news, is emailed to subscribers bya.m. every business day. Sign up for the e mail here. On Friday evening, with Congress out of town on its summer recess and Americans heading into a mid August weekend, the Bush administration sent a message to the states: The federal government will make it tougher for a national childrens insurance program to cover the offspring of middle income families. The State Childrens Health In
J.P. Morgan Chase & Co. reached a tentative deal this weekend to pay $ billion to end a number of civil investigations into its sale of mortgage securities before thefinancial crisis, but a separate and potentially more serious criminal probe into the bank and its executives will continue. The Justice Department, convinced it has strong evidence related to the banks conduct and eager to send a message to Wall Street, rebuffed repeated attempts by J.P. Morgan to settle the criminal investigation

In [12]:
yes_cleaned

0       NEW YORK Yields on most certificates of deposi...
4       NEW YORK Indecision marked the dollars tone, a...
5       Stocks declined, as investors weighed slower t...
9       The U.S. dollar declined against most major fo...
12      James B. Stewart The dreaded "D" word is back ...
17      NEW YORK The dollar sagged yesterday, as news ...
20      The technology industry, an engine of innovati...
21      WASHINGTON While acknowledging that the pace o...
27      WASHINGTON In a sign that inflation remains un...
30      WASHINGTON Consumer credit in July continued t...
31      Consumers boosted their spending only modestly...
32      WASHINGTON The breakdown in deficit reduction ...
36      HEALTH Recession Likely to Boost Government Ou...
39      WASHINGTON The spate of new jobs created by Am...
41      The dollar jumped to one month highs against t...
46      NEW YORK Its time to call a halt to the bond m...
47      Consumers arent stepping up spending at the pa...
48      Never 

In [13]:
#Combine rows into one string for Spacy parsing.
yes_text_combined = reduce(lambda x, y: x+y, yes_cleaned)
yes_text_combined = yes_text_combined.lower()


no_text_combined = reduce(lambda x, y: x+y, no_cleaned)
no_text_combined = no_text_combined.lower()


In [14]:
#Increase max length for nlp.  Then run Spacy parsing on relevant and not relevant
nlp.max_length = 11000000
yes_nlp = nlp(yes_text_combined)
no_nlp = nlp(no_text_combined)

In [15]:
#Convert parsed text to sentences
yes_sents = [[sent,"Relevant"] for sent in yes_nlp.sents]
no_sents = [[sent,"Not_Relevant"] for sent in no_nlp.sents]
sentences = pd.DataFrame(yes_sents + no_sents)
nlp_text =sentences
sentences.head(10)

,0,1
0,"(new, york, yields, on, most, certificates, of...",Relevant
1,"(on, small, denomination, ,, or, "", consumer, ...",Relevant
2,"(on, three, month, "", consumer, "", deposits, ,...",Relevant
3,"(two, banks, in, the, banxquote, survey, ,, ci...",Relevant
4,"(declines, were, somewhat, smaller, on, five, ...",Relevant
5,"(yields, on, three, month, and, six, month, tr...",Relevant
6,"(the, dollar, ended, the, new, york, day, slig...",Relevant
7,"(market, participants, were, also, reluctant, ...",Relevant
8,"(late, yesterday, afternoon, in, new, york, ,,...",Relevant
9,"(cents, ,, slightly, stronger, than, its, ., c...",Relevant


In [18]:
# Create sentences for parts of speech parsing
yes_sentence = list(yes_nlp.sents)
no_sentence = list(no_nlp.sents)
combined_sentence = (yes_sentence + no_sentence)

In [19]:
# Count adverbs
adv_count =[]
for sent in combined_sentence:
    count=0
    for token in sent:
        if token.pos_ == "ADV":
            count = count+1
        else:
            count=count
    adv_count.append(count)

In [20]:
# Count nouns
noun_count =[]
for sent in combined_sentence:
    count=0
    for token in sent:
        if token.pos_ == "NOUN":
            count = count+1
        else:
            count=count
    noun_count.append(count)

In [21]:
# Count adjectives
adj_count =[]
for sent in combined_sentence:
    count=0
    for token in sent:
        if token.pos_ == "ADJ":
            count = count+1
        else:
            count=count
    adj_count.append(count)

In [23]:
# Word counter function
def word_counter(text, include_stop=True):
        words=[]
        for token in text:
            if not token.is_punct and (not token.is_stop or include_stop):
                words.append(token.text)
        return Counter(words)

In [24]:
# Word frequency for relevant text
word_frequency_relevant = word_counter(yes_nlp, include_stop=False).most_common(100)
print(word_frequency_relevant)

[('$', 597), ('said', 567), ('year', 523), ('market', 481), ('u.s', 480), ('prices', 395), ('rates', 367), ('economy', 367), ('rate', 360), ('economic', 339), ('interest', 327), ('stock', 327), ('new', 322), ('federal', 275), ('quarter', 270), ('growth', 263), ('investors', 256), ('billion', 248), ('dollar', 243), ('fed', 238), ('index', 237), ('york', 235), ('nt', 233), ('rose', 227), ('inflation', 227), ('month', 204), ('average', 202), (' ', 202), ('bond', 200), ('stocks', 196), ('fell', 188), ('reserve', 188), ('trading', 172), ('dow', 170), ('higher', 165), ('consumer', 162), ('sales', 157), ('week', 155), ('markets', 155), ('analysts', 152), ('mr', 149), ('department', 147), ('bank', 146), ('industrial', 146), ('jones', 144), ('months', 144), ('companies', 142), ('labor', 141), ('years', 141), ('spending', 140), ('report', 139), ('term', 133), ('banks', 131), ('price', 131), ('increase', 130), ('points', 129), ('financial', 128), ('government', 126), ('data', 125), ('recession', 

In [25]:
# Word frequency for non-relevant
word_frequency_not_relevant = word_counter(no_nlp, include_stop=False).most_common(100)
print(word_frequency_not_relevant)

[('$', 636), ('said', 473), ('u.s', 467), ('market', 441), ('year', 437), ('stock', 364), ('nt', 358), ('new', 322), ('rates', 280), ('mr', 259), ('investors', 252), ('billion', 237), ('interest', 234), ('economic', 228), ('economy', 224), ('federal', 223), ('rate', 219), ('prices', 203), ('million', 197), (' ', 192), ('dollar', 188), ('york', 174), ('markets', 174), ('stocks', 165), ('fed', 165), ('inflation', 161), ('growth', 161), ('companies', 160), ('banks', 154), ('week', 153), ('quarter', 153), ('years', 152), ('says', 146), ('bank', 142), ('reserve', 137), ('government', 132), ('time', 131), ('bond', 131), ('s', 130), ('term', 129), ('trading', 128), ('exchange', 127), ('financial', 125), ('inc', 125), ('company', 124), ('price', 124), ('average', 122), ('money', 122), ('high', 121), ('policy', 114), ('president', 114), ('people', 113), ('funds', 111), ('recent', 110), ('index', 110), ('capital', 109), ('fell', 106), ('trade', 106), ('investment', 106), ('higher', 104), ('bonds

In [26]:
# Function to count lemmas
def lemma_frequencies(text, include_stop=True):
    lemmas = []
    for token in text:
        if not token.is_punct and (not token.is_stop or include_stop):
            lemmas.append(token.lemma_)
    return Counter(lemmas)

In [27]:
#See most common lemmas for relevant text
text_lemma_freq_relevant = lemma_frequencies(yes_nlp, include_stop=False).most_common(10)
print(text_lemma_freq_relevant)

[('rate', 727), ('year', 664), ('say', 658), ('market', 636), ('$', 597), ('price', 530), ('stock', 523), ('rise', 439), ('u.s', 403), ('economy', 385)]


In [28]:
#See most common lemmas
text_lemma_freq_not_relevant = lemma_frequencies(no_nlp, include_stop=False).most_common(10)
print(text_lemma_freq_not_relevant)

[('say', 639), ('$', 636), ('market', 616), ('year', 589), ('stock', 530), ('rate', 499), ('u.s', 380), ('not', 358), ('price', 340), ('new', 326)]


In [29]:
#Bag of words features
def bag_of_words(text):
    allwords = [token.lemma_ for token in text if not token.is_punct and not token.is_stop]
    return [item[0] for item in Counter(allwords).most_common(2000)]

def bow_features(sentences, common_words):
    bow_df = pd.DataFrame(columns=common_words)
    bow_df["text"] = sentences[0]
    bow_df["relevance"] = sentences[1]
    bow_df.loc[:,common_words] = 0
    for i, sentence in enumerate(bow_df["text"]):
        words = [token.lemma_ for token in sentence if (not token.is_punct and not token.is_stop and token.lemma_ in common_words)]
        for word in words:
            bow_df.loc[i, word] +=1
        if i % 500 == 0:
            print("Processing row {}".format(i))
    return bow_df

In [30]:
# Run bag of words on relevant and not relevant sets.  Then combine.
yes_words = bag_of_words(yes_nlp)
no_words = bag_of_words(no_nlp)
common_words = set(yes_words + no_words)

In [31]:
# Add words to dataframe then count for each row of text
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000
Processing row 5500
Processing row 6000
Processing row 6500
Processing row 7000
Processing row 7500
Processing row 8000
Processing row 8500
Processing row 9000
Processing row 9500
Processing row 10000


,story,ally,currently,opinion,schedule,cigarette,well,experiment,blog,spurt,mountain,micro,tender,bargaining,decent,limit,wednesday,face,strength,benefit,list,advice,dec,heavily,freddie,couple,sunday,francisco,florida,infrastructure,partnership,excuse,minus,default,earning,investing,k,sach,speculate,ward,super,gainer,sell,sizable,annually,join,restrict,boehner,repay,recent,murakami,touch,insist,ecb,atlanta,cyclical,competitive,recommendation,electric,red,steadily,brake,fail,especially,provide,bolster,jerry,chairwoman,extraordinary,researcher,tap,considerable,public,slowly,elect,amplification,internal,funding,ownership,resolution,potential,stream,vacancy,iraq,state,fresh,shareholder,role,blackstone,back,bet,violate,belief,roger,contribution,marginal,spark,mix,efficiency,domestic,hundred,send,grade,boost,foreclosure,expensive,restriction,composite,demand,indicator,italy,oil,comptroller,lending,obligation,young,careful,approve,package,adjust,produce,prompt,pullback,writer,assure,yardeni,dive,capital,relationship,maintain,increasingly,movie,friday,b.,sag,lynch,court,repair,overheat,president,slightly,dodge,reit,cleanup,a.m.,concerned,convince,up,collect,december,swing,halve,analyze,cisco,homeowner,go,discount,finally,day,quarter,probably,maturity,charle,ounce,load,sudden,hotel,post,thank,welfare,apparently,basel,grow,erode,thanksgiving,archipelago,difficult,thin,black,telephone,row,cycle,follow,closed,revenue,entity,damp,key,feeling,idaho,greenback,single,involve,watcher,hill,bother,dispute,phase,petroleum,cash,eighth,largely,clean,thousand,john,cds,count,retiree,tell,mexican,challenge,disease,speculation,old,pass,emergency,lay,stand,weight,cheer,policy,bankruptcy,effective,taiwan,different,have,variety,soar,buoy,amid,temporarily,princeton,missouri,simple,question,moody,airway,south,sight,blow,manufacture,donald,skittish,lawmaker,shoot,minneapolis,market,complain,fear,ford,...,because,matter,newswir,dividend,attack,death,persuade,wage,online,adjustment,ge,air,react,race,chance,household,available,influence,gemstar,virtually,osi,merchandise,durable,average,pimco,positive,gallon,massachusetts,illustrate,man,commit,billion,macroeconomic,academic,threat,competitor,discussion,better,kansa,co.,bust,suppose,ready,total,britain,seize,middle,surprisingly,mount,possibility,reasonable,thanpoint,m,u.,faith,schwab,world,step,surge,russia,defense,enact,include,citigroup,mideast,reading,oct,way,continental,upper,root,east,anti,higher,downgrade,closing,curb,constrain,founder,miss,close,weather,recently,today,suggest,corruption,oversee,california,hardly,snap,crimp,baker,ciena,jobless,spill,march,nextyear,rally,southeast,invasion,productivity,edge,wisdom,absorb,takeover,alan,saving,bleak,identify,double,decliner,s.,field,adviser,debt,jack,airline,loosen,home,bar,operating,area,indicate,greatly,reverse,living,nobel,inflow,reflect,overshadow,finance,indication,affect,favorite,depression,assistant,worth,await,flow,taxable,professor,compete,mistake,citizen,number,early,opportunity,negotiation,flat,good,denominate,fashion,need,record,accounting,deposit,stress,opposition,expire,unlike,consistently,trial,lawrence,encouraging,class,firefighter,reject,argument,soft,remarkable,reach,tie,maximum,said.washington,model,microsoft,light,prove,handful,coming,fit,joel,economist,lock,reform,optimist,publisher,party,split,draghi,lose,article,accompany,idea,poise,subsidy,enter,research,productive,automotive,term,self,low,group,prime,burden,track,express,deteriorate,exxon,use,profitability,interest,opening,thought,widely,individual,goal,tiny,journal,technical,ensure,battle,procter,yield,spirit,web,fan,chinese,stephen,absence,walk,creditor,wonder,slow,response,defeat,capitalist,sharp,communication,pastmonth,behavior,analyst,bush,suisse,broad,conclusion,euro,text,relevance
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [32]:
# Add parts of speech counts to BOW dataframe
word_counts["ADJ"] = adj_count
word_counts["NOUN"] = noun_count
word_counts["ADV"] = adv_count

In [33]:
# See final BOW dataframe
word_counts.head()

,story,ally,currently,opinion,schedule,cigarette,well,experiment,blog,spurt,mountain,micro,tender,bargaining,decent,limit,wednesday,face,strength,benefit,list,advice,dec,heavily,freddie,couple,sunday,francisco,florida,infrastructure,partnership,excuse,minus,default,earning,investing,k,sach,speculate,ward,super,gainer,sell,sizable,annually,join,restrict,boehner,repay,recent,murakami,touch,insist,ecb,atlanta,cyclical,competitive,recommendation,electric,red,steadily,brake,fail,especially,provide,bolster,jerry,chairwoman,extraordinary,researcher,tap,considerable,public,slowly,elect,amplification,internal,funding,ownership,resolution,potential,stream,vacancy,iraq,state,fresh,shareholder,role,blackstone,back,bet,violate,belief,roger,contribution,marginal,spark,mix,efficiency,domestic,hundred,send,grade,boost,foreclosure,expensive,restriction,composite,demand,indicator,italy,oil,comptroller,lending,obligation,young,careful,approve,package,adjust,produce,prompt,pullback,writer,assure,yardeni,dive,capital,relationship,maintain,increasingly,movie,friday,b.,sag,lynch,court,repair,overheat,president,slightly,dodge,reit,cleanup,a.m.,concerned,convince,up,collect,december,swing,halve,analyze,cisco,homeowner,go,discount,finally,day,quarter,probably,maturity,charle,ounce,load,sudden,hotel,post,thank,welfare,apparently,basel,grow,erode,thanksgiving,archipelago,difficult,thin,black,telephone,row,cycle,follow,closed,revenue,entity,damp,key,feeling,idaho,greenback,single,involve,watcher,hill,bother,dispute,phase,petroleum,cash,eighth,largely,clean,thousand,john,cds,count,retiree,tell,mexican,challenge,disease,speculation,old,pass,emergency,lay,stand,weight,cheer,policy,bankruptcy,effective,taiwan,different,have,variety,soar,buoy,amid,temporarily,princeton,missouri,simple,question,moody,airway,south,sight,blow,manufacture,donald,skittish,lawmaker,shoot,minneapolis,market,complain,fear,ford,...,dividend,attack,death,persuade,wage,online,adjustment,ge,air,react,race,chance,household,available,influence,gemstar,virtually,osi,merchandise,durable,average,pimco,positive,gallon,massachusetts,illustrate,man,commit,billion,macroeconomic,academic,threat,competitor,discussion,better,kansa,co.,bust,suppose,ready,total,britain,seize,middle,surprisingly,mount,possibility,reasonable,thanpoint,m,u.,faith,schwab,world,step,surge,russia,defense,enact,include,citigroup,mideast,reading,oct,way,continental,upper,root,east,anti,higher,downgrade,closing,curb,constrain,founder,miss,close,weather,recently,today,suggest,corruption,oversee,california,hardly,snap,crimp,baker,ciena,jobless,spill,march,nextyear,rally,southeast,invasion,productivity,edge,wisdom,absorb,takeover,alan,saving,bleak,identify,double,decliner,s.,field,adviser,debt,jack,airline,loosen,home,bar,operating,area,indicate,greatly,reverse,living,nobel,inflow,reflect,overshadow,finance,indication,affect,favorite,depression,assistant,worth,await,flow,taxable,professor,compete,mistake,citizen,number,early,opportunity,negotiation,flat,good,denominate,fashion,need,record,accounting,deposit,stress,opposition,expire,unlike,consistently,trial,lawrence,encouraging,class,firefighter,reject,argument,soft,remarkable,reach,tie,maximum,said.washington,model,microsoft,light,prove,handful,coming,fit,joel,economist,lock,reform,optimist,publisher,party,split,draghi,lose,article,accompany,idea,poise,subsidy,enter,research,productive,automotive,term,self,low,group,prime,burden,track,express,deteriorate,exxon,use,profitability,interest,opening,thought,widely,individual,goal,tiny,journal,technical,ensure,battle,procter,yield,spirit,web,fan,chinese,stephen,absence,walk,creditor,wonder,slow,response,defeat,capitalist,sharp,communication,pastmonth,behavior,analyst,bush,suisse,broad,conclusion,euro,text,relevance,ADJ,NOUN,ADV
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [34]:
#Columsn for BOW
columns=word_counts.columns
for line in columns:
    print(line)

story
ally
currently
opinion
schedule
cigarette
well
experiment
blog
spurt
mountain
micro
tender
bargaining
decent
limit
wednesday
face
strength
benefit
list
advice
dec
heavily
freddie
couple
sunday
francisco
florida
infrastructure
partnership
excuse
minus
default
earning
investing
k
sach
speculate
ward
super
gainer
sell
sizable
annually
join
restrict
boehner
repay
recent
murakami
touch
insist
ecb
atlanta
cyclical
competitive
recommendation
electric
red
steadily
brake
fail
especially
provide
bolster
jerry
chairwoman
extraordinary
researcher
tap
considerable
public
slowly
elect
amplification
internal
funding
ownership
resolution
potential
stream
vacancy
iraq
state
fresh
shareholder
role
blackstone
back
bet
violate
belief
roger
contribution
marginal
spark
mix
efficiency
domestic
hundred
send
grade
boost
foreclosure
expensive
restriction
composite
demand
indicator
italy
oil
comptroller
lending
obligation
young
careful
approve
package
adjust
produce
prompt
pullback
writer
assure
yardeni
di

compromise
eye
demonstrate
hybrid
harsh
kong
alcoa
active
struggle
kudlow
shopping
insurer
google
data
direction
offset
recovery
somewhat
adelphia
appeal
political
fraction
gathering
widen
family
stir
assert
adobe
bull
source
conundrum
mixed
partly
schorsch
tomorrow
rapid
person
rebound
brace
extremely
lead
straight
art
hang
crist
chief
parker
arrive
common
operator
spokesman
specialist
few
disappointment
moment
prepare
hopeful
place
clinton
trip
mellon
site
philadelphia
asset
loehmann
weakness
proposal
supply
path
inflationary
crash
narrow
get
comeback
june
fla
history
jan
wrong
overall
europe
woman
effort
spending
bounce
borrower
entire
situation
specter
secretary
stimulate
ltd
attempt
inand
g
western
coldwell
genetic
triple
rattle
yen
medicare
conference
program
central
editor
tuesday
legislative
zero
creek
passenger
august
school
era
scrutiny
tightening
comparison
point
simply
unemployment
producer
version
manufacturer
unable
bureau
drug
testing
nominal
auction
case
valuation
effic

claremont
form
dome
nikkei
unit
carrington
clip
prior
calculation
consulting
regular
commissioner
nation
target
.washington
like
cause
dramatically
understand
suspect
pincus
marquez
initially
loan
gainedcent
catch
restore
head
knowledge
climb
rating
postpone
status
democratic
fall
part
popular
pricing
congressional
advantage
cap
address
nationwide
indian
merrill
silver
plague
cautious
obama
view
strike
alike
concession
easily
pilot
push
lean
million
nonfarm
deserve
gao
differ
disappoint
horizon
mark
originally
july
figure
reduce
final
recoup
engineer
typically
fargo
retreat
p.m.
ticket
time
strategist
circuit
security
imminent
perk
seventh
implication
long
disclose
rosecent
outlay
watch
king
equipment
special
take
administration
holiday
damage
general
federal
inyear
w.
sale
revision
iraqs
provider
pre
assume
tower
answer
terrorist
drag
plenty
commerce
chip
nice
nov
folk
explain
maker
because
matter
newswir
dividend
attack
death
persuade
wage
online
adjustment
ge
air
react
race
chance
h

In [35]:
#Random forest model function
def random_forest_model(x_train,y_train,x_test,y_test, random_state=42):
    rfc = ensemble.RandomForestClassifier()
    model = rfc.fit(x_train, y_train)
    print('Training set score:', model.score(x_train, y_train))
    print('\nTest set score:', model.score(x_test, y_test))
    cross = cross_val_score(rfc,x_test, y_test, cv=3)
    print(cross)
    

In [36]:
# Bag of words train-test-split

y = word_counts['relevance']
x = np.array(word_counts.drop(['text','relevance'], 1))

x_train_bow, x_test_bow, y_train_bow, y_test_bow = train_test_split(x, y, test_size=0.4, random_state=0)

In [37]:
#Random forest model function
def random_forest_model(estimators,x_train,y_train,x_test,y_test, random_state=42):
    rfc = ensemble.RandomForestClassifier(n_estimators=estimators)
    model = rfc.fit(x_train, y_train)
    print('Training set score:', model.score(x_train, y_train))
    print('\nTest set score:', model.score(x_test, y_test))
    cross = cross_val_score(rfc,x_test, y_test, cv=3)
    print(cross)

In [67]:
# Run random forest on bag of words features
random_forest_model(500,x_train_bow, y_train_bow, x_test_bow, y_test_bow)

Training set score: 0.983765717014

Test set score: 0.635560859189
[ 0.62160229  0.60243553  0.6217765 ]


In [39]:
#Create vectorizer with train-test-split

vectorizer = TfidfVectorizer(max_df=0.5, min_df=2, stop_words='english', lowercase=True, use_idf=True, norm=u'l2', smooth_idf=True)

vector_x_train, vector_x_test, vector_y_train, vector_y_test = train_test_split(cleaned_text, relevance, test_size=0.4, random_state=0)

In [40]:
#Vectorizer fit and transform.
text_tfidf = vectorizer.fit_transform(cleaned_text)

x_train_tfidf, x_test_tfidf = train_test_split(text_tfidf, test_size=.4, random_state=42)

x_train_tfidf_csr = x_train_tfidf.tocsr()

n = x_train_tfidf_csr.shape[0]

tfidf_bypara = [{} for _ in range(0,n)]

terms = vectorizer.get_feature_names()

for i, j in zip(*x_train_tfidf_csr.nonzero()):
    tfidf_bypara[i][terms[j]] = x_train_tfidf_csr[i, j]


In [68]:
#SVD to reduce features created by LSA and vectorizer
svd = TruncatedSVD(1000)
lsa = make_pipeline(svd, Normalizer(copy=False))
# Run SVD on the training data, then project the training data.
x_train_lsa = lsa.fit_transform(x_train_tfidf)

variance_explained=svd.explained_variance_ratio_
total_variance = variance_explained.sum()
print("Percent variance captured by all components:",total_variance*100)

#Looking at what sorts of paragraphs our solution considers similar, for the first five identified topics
paras_by_component=pd.DataFrame(x_train_lsa,index=vector_x_train)


Percent variance captured by all components: 59.0889351192


In [43]:
#Create x train dataframe with features for modeling
vector_x_train_transformed = pd.DataFrame(x_train_lsa)

#Transform x_test
x_test_lsa = lsa.transform(x_test_tfidf)

#Create dataframe with x test transformed features
vector_x_test_transformed = pd.DataFrame(x_test_lsa)

In [44]:
vector_x_train_transformed.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,830,831,832,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,867,868,869,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
0,0.315755,0.180203,0.121496,-0.021420,0.038863,0.018789,0.067352,-0.004774,-0.077368,-0.070513,-0.035019,0.181071,0.000987,0.052596,-0.121934,0.025222,0.037799,0.021663,-0.100636,-0.002513,0.082750,0.024297,-0.027812,-0.025178,0.032041,-0.065609,0.006091,-0.017471,-0.018233,-0.038412,0.012407,-0.038488,-0.056790,0.038052,-0.015903,-0.014796,-0.025074,0.084105,-0.001143,-0.079526,0.029768,-0.027199,0.059906,-0.031055,-0.026961,0.047740,0.013285,0.037731,0.004816,-0.033982,0.003753,0.000793,-0.089972,0.017179,-0.019926,0.071127,0.025716,-0.006197,0.020942,-0.041729,0.040064,0.034448,0.056959,0.045079,-0.017773,0.044769,-0.030499,0.008410,-0.092383,0.062320,-0.094267,-0.015370,0.014023,0.032677,-0.046150,0.003862,0.030453,-0.032244,-0.006877,0.046552,-0.023398,0.013387,-0.037596,0.013924,0.035644,-0.002455,-0.024564,0.066535,-0.062437,-0.072253,-0.010239,-0.022599,-0.008409,0.039681,0.003887,0.020319,0.005284,-0.010026,-0.010471,-0.030156,-0.022968,-0.078673,-0.032272,-0.030517,0.005280,0.011793,0.041985,-0.050138,-0.013844,-0.065682,0.034926,-0.002745,-0.036466,-0.059602,-0.014259,-0.020749,0.020858,-0.038813,-0.007798,-0.007128,0.029480,0.053488,0.042275,-0.004227,0.070033,0.050474,0.042237,-0.003212,-0.037858,0.030814,0.064522,-0.055576,-0.004354,0.014075,-0.003842,-0.050752,-0.040814,-0.043063,-0.062354,0.007196,-0.027126,0.040481,0.028080,-0.013959,-0.096425,0.010979,0.020802,0.011744,0.057510,-0.061139,-0.039254,0.006622,-0.035201,0.009436,0.088255,0.038511,0.014879,-0.006121,-0.023349,0.000693,0.025123,0.016032,0.018004,0.020229,0.056099,0.012689,-0.026093,0.039883,0.003550,0.048068,-0.001296,0.050760,0.006203,-0.075466,0.008846,0.002541,-0.028321,-0.057131,0.045449,-0.025296,0.023552,0.034532,0.005714,0.010576,-0.052753,-0.031467,-0.045169,0.014558,0.017056,-0.067152,-0.004095,-0.004246,0.046215,-0.022700,-0.011494,-0.036043,0.044152,0.035650,0.020708,-0.029187,-0.045259,-0.081982,0.024730,0.049020,-0.037845,-0.020268,0.048078,-0.009526,0.008380,-0.019719,-0.012559,-0.029664,0.010217,-0.018704,0.052404,0.015635,0.032188,0.025618,0.006300,-0.007231,0.023097,0.0125

In [45]:
# Random Forest on vectorizer features

random_forest_model(10,vector_x_train_transformed,vector_y_train,vector_x_test_transformed,vector_y_test)

Training set score: 0.970625

Test set score: 0.81875
[ 0.8153702   0.81349578  0.81425891]


In [51]:
from sklearn.linear_model import LogisticRegression

In [52]:
#Logistic Regression model function
def logistic_model(x_train,y_train,x_test,y_test):
    lr = LogisticRegression()
    train = lr.fit(x_train, y_train)
    print(x_train.shape, y_train.shape)
    print('Training set score:', lr.score(x_train, y_train))
    print('\nTest set score:', lr.score(x_test, y_test))

In [53]:
# Logistic Regression on bag of words features
logistic_model(x_train_bow, y_train_bow, x_test_bow, y_test_bow)

(6283, 2546) (6283,)
Training set score: 0.81569314022

Test set score: 0.645584725537


In [54]:
# Logistic Regression on vectorizer features
logistic_model(vector_x_train_transformed,vector_y_train,vector_x_test_transformed,vector_y_test)

(4800, 1000) (4800,)
Training set score: 0.821458333333

Test set score: 0.8221875


In [55]:
from sklearn.neighbors import KNeighborsClassifier

In [56]:
#KNN Model
def knn_model(x_train,y_train,x_test,y_test):
    knn = KNeighborsClassifier()
    model = knn
    model = model.fit(x_train, y_train)
    print('Training set score:', model.score(x_train, y_train))
    print('\nTest set score:', model.score(x_test, y_test))
    cross = cross_val_score(knn,x_test, y_test, cv=3)
    print(cross)

In [57]:
# KNN on bag of words features
knn_model(x_train_bow, y_train_bow, x_test_bow, y_test_bow)

Training set score: 0.708260385166

Test set score: 0.549880668258
[ 0.53505007  0.52578797  0.5752149 ]


In [58]:
# KNN on vectorizer features
knn_model(vector_x_train_transformed,vector_y_train,vector_x_test_transformed,vector_y_test)

Training set score: 0.8275

Test set score: 0.795625
[ 0.78444236  0.78537957  0.80206379]


In [59]:
from sklearn import ensemble

In [60]:
gradient = ensemble.GradientBoostingClassifier()

In [61]:
#Logistic Regression model function
def gradient_model(x_train,y_train,x_test,y_test):
    g = ensemble.GradientBoostingClassifier()
    train = g.fit(x_train, y_train)
    print('Training set score:', g.score(x_train, y_train))
    print('\nTest set score:', g.score(x_test, y_test))

In [62]:
# Gradient Model on bag of words features
gradient_model(x_train_bow, y_train_bow, x_test_bow, y_test_bow)

Training set score: 0.690911984721

Test set score: 0.614319809069


In [63]:
# Gradient model on vectorizer features
gradient_model(vector_x_train_transformed,vector_y_train,vector_x_test_transformed,vector_y_test)

Training set score: 0.842291666667

Test set score: 0.8196875


In [64]:
from sklearn.naive_bayes import GaussianNB

In [69]:
#KNN Model
def nb_model(x_train,y_train,x_test,y_test):
    nb = KNeighborsClassifier()
    model = nb
    model = model.fit(x_train, y_train)
    print('Training set score:', model.score(x_train, y_train))
    print('\nTest set score:', model.score(x_test, y_test))
    cross = cross_val_score(nb,x_test, y_test, cv=3)
    print(cross)

In [70]:
# Naive Bayes on bag of words features
nb_model(x_train_bow, y_train_bow, x_test_bow, y_test_bow)

Training set score: 0.708260385166

Test set score: 0.549880668258
[ 0.53505007  0.52578797  0.5752149 ]


In [71]:
# Naive bayes model on vectorizer features
nb_model(vector_x_train_transformed,vector_y_train,vector_x_test_transformed,vector_y_test)

Training set score: 0.8275

Test set score: 0.795625
[ 0.78444236  0.78537957  0.80206379]
